De code is zodanig geschreven dat je van bovenaf moet beginnen om alle definities, benamingen,... mee te hebben in de latere code.
Maar ook dat er maar 1 plaats is waar je tabelnamen moet aanpassen.

de parameters dienen om de code sneller te laten lopen als je gewoon wilt testen omdat je dan ook nog steeds de hele code moet laten lopen.
Je kan dit in principe vermijden door elke keer uit de SQL tabellen te gaan lezen, maar dat maakt de code ook enorm, enorm traag

In [1]:
%run Fnc_StartupFabricSempy

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 9, Finished, Available, Finished)

In [2]:
prm_starttime = datetime.now()
NotebookName = "Nbt_Landing_GetUsers"

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 10, Finished, Available, Finished)

In [3]:
Table_Name = 'Landing_Fabric_Users'

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 11, Finished, Available, Finished)

In [4]:
lh_abfs_path = fnc_startUp(Table_Name)

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 12, Finished, Available, Finished)

niet leeg


SynapseWidget(Synapse.DataFrame, f444f3b5-0f3f-41ec-8716-85101531014f)

In [5]:
Workspaces = spark.sql("""select Id, Name
from Lkh_SemanticLink.Landing_Fabric_Workspaces""")

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 13, Finished, Available, Finished)

In [6]:
for Id, Name in Workspaces.toLocalIterator():
    try:
        temp_items = sempy_labs.list_workspace_users(workspace=Id)
        itemdf = pd.DataFrame(temp_items)
    except Exception as e:
        print(e)
    if not itemdf.empty: # check if the list is not empty to avoid errors
    #prepare items and write them away
        try:
            itemdf = fnc_PrepareColumns(itemdf)
            sparkdf = spark.createDataFrame(itemdf)
            sparkdf = sparkdf.withColumn('WSID', lit(Id))
            sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")
            #print(Table_Name_Items, "created at :", f"{lh_abfs_path}/Tables/{Table_Name_Items}")                        
        except Exception as e:
            print(f"Error fetching Workspace objects for {Id}: {e}")
            continue

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 14, Finished, Available, Finished)

In [4]:
import pyspark
from pyspark.sql import SparkSession

#fill list of workspaces
workspaces = fabric.list_workspaces().sort_values(by='Name', ascending=True)
df = pd.DataFrame(workspaces)

# loop through the workspaces
for label, content in df.items():
    if label == 'Id': #loop through the ID's
        for id in content:
            #lijst vullen met alle items in de werkruimte
            try:
                temp_items = sempy_labs.list_workspace_users(workspace=id)
                itemdf = pd.DataFrame(temp_items)
            except Exception as e:
                print(e)
            if not itemdf.empty: # check if the list is not empty to avoid errors
                #prepare items and write them away
                try:
                    itemdf = fnc_PrepareColumns(itemdf)
                    sparkdf = spark.createDataFrame(itemdf)
                    sparkdf = sparkdf.withColumn('WSID', lit(id))
                    sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")
                    #print(Table_Name_Items, "created at :", f"{lh_abfs_path}/Tables/{Table_Name_Items}")                        
                except Exception as e:
                    print(f"Error fetching Workspace objects for {id}: {e}")
                    continue

StatementMeta(, 09a34f69-ee02-415f-bbec-940a5f501af3, 11, Finished, Available, Finished)

In [7]:
prm_endtime = datetime.utcnow()
fnc_LognotebookRefresh(NotebookName, prm_starttime, prm_endtime)

StatementMeta(, 15d4f2c2-c5fe-48aa-a78e-73470703dfd6, 15, Finished, Available, Finished)